In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plot
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import datetime
from datetime import timedelta

from config import api_key

In [3]:
weather_data_file = pd.read_csv('weather-data-2013-2017.csv')
df = pd.DataFrame(weather_data_file)
df.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1356998400,2013-01-01 00:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,37.33,31.33,34.76,37.99,...,210,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
1,1357002000,2013-01-01 01:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,36.95,30.15,34.90,37.94,...,170,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
2,1357005600,2013-01-01 02:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,36.84,28.78,34.82,37.94,...,190,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
3,1357009200,2013-01-01 03:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,36.34,29.98,33.80,37.40,...,180,0.5,0.65,NaN,NaN,90,500,Rain,light rain,10n
4,1357012800,2013-01-01 04:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,35.33,27.91,33.80,36.00,...,180,0.8,NaN,NaN,NaN,90,501,Rain,moderate rain,10n


In [4]:
# start stop and step variables
start, stop, step = 0, -22, 1

# converting to string data type 
df["dt_iso"]= df["dt_iso"].astype(str) 

# slicing till month element 
df["Date"]= df["dt_iso"].str.slice(start, stop, step) 

# display 
df.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,Date
0,1356998400,2013-01-01 00:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,37.33,31.33,34.76,37.99,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2013-01
1,1357002000,2013-01-01 01:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,36.95,30.15,34.90,37.94,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2013-01
2,1357005600,2013-01-01 02:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,36.84,28.78,34.82,37.94,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2013-01
3,1357009200,2013-01-01 03:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,36.34,29.98,33.80,37.40,...,0.5,0.65,NaN,NaN,90,500,Rain,light rain,10n,2013-01
4,1357012800,2013-01-01 04:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,35.33,27.91,33.80,36.00,...,0.8,NaN,NaN,NaN,90,501,Rain,moderate rain,10n,2013-01


In [5]:
df.columns

Index(['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'sea_level',
       'grnd_level', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h',
       'rain_3h', 'snow_1h', 'snow_3h', 'clouds_all', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon', 'Date'],
      dtype='object')

In [6]:
data = df.rename(columns={"wind_deg": "Wind", "weather_icon": "weather_icon",
                                  "temp": "Temperature", "temp_min": "Minimum Temperature", "temp_max": "Maximum Temperature",
                                  "weather_description": "Weather Description", "clouds_all": "Clouds",
                         "wind_speed": "Wind Speed", "weather_icon": "Weather Icon"})

data.head()

,dt,dt_iso,timezone,city_name,lat,lon,Temperature,feels_like,Minimum Temperature,Maximum Temperature,...,rain_1h,rain_3h,snow_1h,snow_3h,Clouds,weather_id,weather_main,Weather Description,Weather Icon,Date
0,1356998400,2013-01-01 00:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,37.33,31.33,34.76,37.99,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2013-01
1,1357002000,2013-01-01 01:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,36.95,30.15,34.90,37.94,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2013-01
2,1357005600,2013-01-01 02:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,36.84,28.78,34.82,37.94,...,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n,2013-01
3,1357009200,2013-01-01 03:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,36.34,29.98,33.80,37.40,...,0.5,0.65,NaN,NaN,90,500,Rain,light rain,10n,2013-01
4,1357012800,2013-01-01 04:00:00 +0000 UTC,-18000,Louisville,38.252665,-85.758456,35.33,27.91,33.80,36.00,...,0.8,NaN,NaN,NaN,90,501,Rain,moderate rain,10n,2013-01


In [7]:
# data['Date'] = pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45354 entries, 0 to 45353
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   dt                   45354 non-null  int64  
 1   dt_iso               45354 non-null  object 
 2   timezone             45354 non-null  int64  
 3   city_name            45354 non-null  object 
 4   lat                  45354 non-null  float64
 5   lon                  45354 non-null  float64
 6   Temperature          45354 non-null  float64
 7   feels_like           45354 non-null  float64
 8   Minimum Temperature  45354 non-null  float64
 9   Maximum Temperature  45354 non-null  float64
 10  pressure             45354 non-null  int64  
 11  sea_level            0 non-null      float64
 12  grnd_level           0 non-null      float64
 13  humidity             45354 non-null  int64  
 14  Wind Speed           45354 non-null  float64
 15  Wind                 45354 non-null 

In [8]:
data = data.drop(columns=['dt', 'dt_iso','timezone', 'city_name',
                          'pressure', 'sea_level', 'grnd_level', 'humidity',
                         'pressure','sea_level', 'grnd_level', 'humidity',
                         'rain_1h','rain_3h', 'snow_1h', 'snow_3h','weather_id', 'weather_main'])
data.head()

,lat,lon,Temperature,feels_like,Minimum Temperature,Maximum Temperature,Wind Speed,Wind,Clouds,Weather Description,Weather Icon,Date
0,38.252665,-85.758456,37.33,31.33,34.76,37.99,3.36,210,90,overcast clouds,04n,2013-01
1,38.252665,-85.758456,36.95,30.15,34.90,37.94,4.70,170,90,overcast clouds,04n,2013-01
2,38.252665,-85.758456,36.84,28.78,34.82,37.94,6.93,190,90,overcast clouds,04n,2013-01
3,38.252665,-85.758456,36.34,29.98,33.80,37.40,4.70,180,90,light rain,10n,2013-01
4,38.252665,-85.758456,35.33,27.91,33.80,36.00,6.93,180,90,moderate rain,10n,2013-01


In [9]:
year_group = data.groupby(["Date","lat","lon"]).agg({'Maximum Temperature': ['mean']}).reset_index()


year_group.head()


,Date,lat,lon,Maximum Temperature
,,,,mean
0,2013-01,38.252665,-85.758456,39.235639
1,2013-02,38.252665,-85.758456,38.266915
2,2013-03,38.252665,-85.758456,41.918568
3,2013-04,38.252665,-85.758456,59.849450
4,2013-05,38.252665,-85.758456,69.487237


In [10]:
year_group.columns= ['Year/Month', 'Latitude', 'Longitude', 'Max Temperature (mean)']

year_group.head()

,Year/Month,Latitude,Longitude,Max Temperature (mean)
0,2013-01,38.252665,-85.758456,39.235639
1,2013-02,38.252665,-85.758456,38.266915
2,2013-03,38.252665,-85.758456,41.918568
3,2013-04,38.252665,-85.758456,59.849450
4,2013-05,38.252665,-85.758456,69.487237


In [11]:
# year_group = data.groupby("Date")['Temperature'].mean()
# # year_group
# mean_temp = pd.DataFrame(year_group).reset_index()
# meantemp = mean_temp.rename(columns={'Temperature': 'Mean Temperature'})
# meantemp.head()

In [12]:
gmaps.configure(api_key=api_key)

locations = year_group[["Latitude","Longitude"]]
Temp = year_group['Max Temperature (mean)'].astype(float)
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=Temp))
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
animal_bites = 'bites_to_weather.csv'
bite_data = pd.read_csv(animal_bites)

bite_data

,Unnamed: 0,Date,Animal Type,Breed,Gender,Victim Zip Code,Where Bitten,Year
0,0,1985-05-05,DOG,NaN,FEMALE,40229,BODY,1985.0
1,1,1986-02-12,DOG,NaN,UNKNOWN,40218,BODY,1986.0
2,2,1987-05-07,DOG,NaN,UNKNOWN,40219,BODY,1987.0
3,3,1988-10-02,DOG,NaN,MALE,NaN,BODY,1988.0
4,4,1989-08-29,DOG,NaN,FEMALE,NaN,BODY,1989.0
...,...,...,...,...,...,...,...,...
8998,8998,2017-09-05,DOG,NaN,NaN,40243,UNKNOWN,2017.0
8999,8999,2017-09-07,DOG,POMERANIAN,MALE,40204,HEAD,2017.0
9000,9000,2017-09-07,DOG,LABRADOR RETRIV,MALE,47130,UNKNOWN,2017.0
9001,9001,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,40229,BODY,2017.0


In [32]:
#need the year and month
start, stop, step = 0, -3, 1

bite_data["Date"]= bite_data["Date"].astype(str) 


bite_data["Year/Month"]= bite_data["Date"].str.slice(start, stop, step) 

bite_data = bite_data.drop(columns='Date')

bite_data = bite_data.sort_values('Gender')

bite_data

,Unnamed: 0,Animal Type,Breed,Gender,Victim Zip Code,Where Bitten,Year,Year/Month
0,0,DOG,NaN,FEMALE,40229,BODY,1985.0,1985-05
4672,4672,DOG,NaN,FEMALE,40245,BODY,2013.0,2013-09
4674,4674,DOG,LABRADOR RETRIV,FEMALE,40291,BODY,2013.0,2013-09
4675,4675,DOG,PIT BULL,FEMALE,40203,BODY,2013.0,2013-09
4680,4680,DOG,PIT BULL,FEMALE,40212,NaN,NaN,
...,...,...,...,...,...,...,...,...
8984,8984,CAT,NaN,NaN,40291,UNKNOWN,2017.0,2017-09
8992,8992,DOG,PIT BULL,NaN,40207,BODY,2017.0,2017-09
8996,8996,DOG,NaN,NaN,40214,BODY,2017.0,2017-09
8998,8998,DOG,NaN,NaN,40243,UNKNOWN,2017.0,2017-09


In [38]:
bite_data["Year"] = bite_data["Year"].astype(str)

data_year_filtered = bite_data[(bite_data['Year'] >= "2013.0") & (bite_data['Year'] <= "2017.0")]

data_year_filtered

,Unnamed: 0,Animal Type,Breed,Gender,Victim Zip Code,Where Bitten,Year,Year/Month
4672,4672,DOG,NaN,FEMALE,40245,BODY,2013.0,2013-09
4674,4674,DOG,LABRADOR RETRIV,FEMALE,40291,BODY,2013.0,2013-09
4675,4675,DOG,PIT BULL,FEMALE,40203,BODY,2013.0,2013-09
4688,4688,CAT,NaN,FEMALE,40203,BODY,2013.0,2013-08
4690,4690,CAT,NaN,FEMALE,NaN,BODY,2013.0,2013-09
...,...,...,...,...,...,...,...,...
8984,8984,CAT,NaN,NaN,40291,UNKNOWN,2017.0,2017-09
8992,8992,DOG,PIT BULL,NaN,40207,BODY,2017.0,2017-09
8996,8996,DOG,NaN,NaN,40214,BODY,2017.0,2017-09
8998,8998,DOG,NaN,NaN,40243,UNKNOWN,2017.0,2017-09
